In [1]:
import random, time
from IPython.display import clear_output

In [6]:
class Card:
    def __init__(self, suit = "", rank = "", value = 0):
        self.suit = suit
        self.rank = rank
        self.value = value
    def __str__(self):
        return f"{self.rank} of {self.suit}"

In [7]:
class Deck:
    def __init__(self, suits, ranks):
        self.cards = []
        for suit in suits:
            for rank, value in ranks.items():
                card = Card(suit, rank, value)
                self.cards.append(card)
    def __str__(self):
        string = ""
        for position_in_deck, card in enumerate(self.cards):
            string += f"{position_in_deck + 1}. {str(card)}\n"
        return string
    def add_cards(self, cards):
        if isinstance(cards, list):
            self.cards.extend(cards)
        elif isinstance(cards, Card):  
            self.cards.append(cards)
    def shuffle(self):
        random.shuffle(self.cards)

In [8]:
class Hand:
    def __init__(self):
        self.cards = []
        self.value = 0
    def __str__(self):
        string = ""
        if self.cards == []:
            string += "Your hand is empty\n"
        else:
            string += "Your hand\n"
            for position_in_hand, card in enumerate(self.cards):
                string += f"{position_in_hand + 1}. {str(card)}\n"
        string += "--------------\n"
        if self.value > 21:
            string += f"Hand value: \033[31m{self.value}\033[0m\n\n"
        else:
            string += f"Hand value: \033[32m{self.value}\033[0m\n\n"
        return string
    def add_cards(self, cards):
        if isinstance(cards, list):
            self.cards.extend(cards)
            for card in cards:
                self.value += card.value
        elif isinstance(cards, Card):  
            self.cards.append(cards)
            self.value += cards.value
    def discard_to(self, deck):
        if isinstance(deck, Deck):
            deck.add_cards(self.cards)
            self.cards = []
            self.value = 0

In [9]:
class Dealer(Hand):
    def __init__(self):
        Hand.__init__(self)
        self.hide_card = True
    def __str__(self):
        string = ""
        if len(self.cards) == 0:
            string += "Dealer's hand is empty"
        else:
            string += "Dealer's hand\n"
            for position_in_hand, card in enumerate(self.cards):
                if position_in_hand == 0 and self.hide_card == True:
                    string += f"{position_in_hand + 1}. [Hidden Card]\n"
                else:
                    string += f"{position_in_hand + 1}. {str(card)}\n"
        string += "--------------\n"
        if self.value > 21:
            string += f"Hand value: \033[31m{self.value}\033[0m\n\n"
        else:
            if self.hide_card == True:
                if len(self.cards) != 0:
                    string += f"Hand value: \033[32m{self.value - self.cards[0].value} + [?]\033[0m\n\n"
                else:
                    string += f"Hand value: \033[32m0\033[0m\n\n"
            else:
                string += f"Hand value: \033[32m{self.value}\033[0m\n\n"
        return string
    def toggle_card_hide(self, toggle_to):
        self.hide_card = toggle_to

In [10]:
class Chips:
    def __init__(self, chips = 100):
        self.chips = chips
        self.betting_amount = 0
    def __str__(self):
        return f"Your chips: \033[32m{self.chips - self.betting_amount}\033[0m chips \nYour bet: \033[32m{self.betting_amount}\033[0m chips \n\n"
    def bet(self, amount):
        try:
            if int(amount) > self.chips or int(amount) < 0:
                print("You don't have enough chips for that bet")
                return False
            else:
                self.betting_amount = int(amount)
                return True
        except:
            print("That's not a valid amount")
            return False
    def reset_bet(self):
        self.betting_amount = 0
    def update(self, round_won):
        if round_won == True:
            print("Round won\n\n")
            self.chips += self.betting_amount
        elif round_won == False:
            print("Round lost\n\n")
            self.chips -= self.betting_amount
        else:
            print("Round tied\n\n")
        self.reset_bet()

In [11]:
class Game:
    def __init__(self):
        self.game_over = False
        self.round = 1
        self.suits = ("\033[31m♥\033[0m", "\033[31m♦\033[0m", "♣", "♠")
        self.ranks = {"Two": 2, "Three": 3, "Four": 4, "Five": 5, "Six": 6, "Seven": 7, "Eight": 8, "Nine": 9, "Ten": 10, "Jack": 10, "Queen": 10, "King": 10, "Ace": 11}
        self.deck = Deck(self.suits, self.ranks)
        self.player_hand = Hand()
        self.player_chips = Chips()
        self.dealer_hand = Dealer()
        self.start()
    def bet(self):
        bet_success = False
        while not bet_success:
            amount = input("Please enter your bet: ")
            bet_success = self.player_chips.bet(amount)
    def deal_hands(self):
        for _ in range(2):
            self.player_hand.add_cards(self.deck.cards.pop(0))
            self.dealer_hand.add_cards(self.deck.cards.pop(0))
    def discard_hands(self):
        self.player_hand.discard_to(self.deck)
        self.dealer_hand.discard_to(self.deck)
    def hit(self):
        card_deal_success = False
        while not card_deal_success:
            deal_response = input("Do you want to Stand (S) or Hit (H)? ")
            if deal_response.lower() == "s":
                card_deal_success = True
                return False
            elif deal_response.lower() == "h":
                card = self.deck.cards.pop(0)
                self.player_hand.add_cards(card)
                card_deal_success = True
                return True
            else:
                card_deal_success = False
    def dealer_turn(self):
        while self.dealer_hand.value < 17:
            self.dealer_hand.add_cards(self.deck.cards.pop(0))
        self.dealer_hand.toggle_card_hide(False)
    def round_won(self):
        self.player_chips.update(True)
    def round_tied(self):
        self.player_chips.update(None)
    def round_lost(self):
        self.player_chips.update(False)
    def is_round_over():
        pass
    def print_chips(self):
        time.sleep(0.5)
        print(self.player_chips)
    def print_table(self):
        time.sleep(0.5)
        clear_output()
        print(f"Round {self.round}\n\n")
        print(self.dealer_hand)
        print(self.player_hand)
    def start(self):
        while not self.game_over:
            self.deck.shuffle()
            self.print_chips()
            self.bet()
            self.deal_hands()
            self.dealer_hand.toggle_card_hide(True)
            self.print_table()
            self.print_chips()
            while self.hit():
                if self.player_hand.value > 21:
                    self.print_table()
                    self.round_lost()
                    break
                else:
                    self.print_table()
                    self.print_chips()
            if self.player_hand.value > 21:
                self.discard_hands()
            else:
                self.dealer_turn()
                self.print_table()
                if self.dealer_hand.value > 21 or self.player_hand.value > self.dealer_hand.value:
                    self.round_won()
                elif self.dealer_hand.value == self.player_hand.value:
                    self.round_tied()
                else:
                    self.round_lost()
                self.discard_hands()
            if self.player_chips.chips == 0:
                print(f"Game Over! \nThis game lasted {self.round} rounds.")
                self.game_over = True
            self.round += 1

In [ ]:
game = Game()
game.start()

Round 4


Dealer's hand
1. [Hidden Card]
2. Five of ♣
--------------
Hand value: 5 + [?]


Your hand
1. Five of ♦
2. Eight of ♥
3. Six of ♥
--------------
Hand value: 19


